## Vacation Search

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os

#sys.path.append('../config.py')
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/valentina/Desktop/Data Visualization/World_Weather_Analysis/weather_data/WeatherPy_Database.csv")
city_data_df.head() 

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nantucket,US,2021-09-25 02:42:23,41.2835,-70.0995,71.96,94,20,12.66
1,1,Alofi,NU,2021-09-25 02:39:25,-19.0595,-169.9187,75.09,83,27,10.36
2,2,Mar Del Plata,AR,2021-09-25 02:34:50,-38.0023,-57.5575,55.13,59,0,1.99
3,3,Yumen,CN,2021-09-25 02:46:32,40.2833,97.2000,57.97,40,9,15.93
4,4,Portland,US,2021-09-25 02:36:17,45.5234,-122.6762,81.18,42,1,1.01


In [6]:
# confirm that numerical columns are float or integer data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What's the minimum temperature you'd like for your trip? "))
max_temp = float(input("What's the maximum temperature you'd like for your trip? "))

What's the minimum temperature you'd like for your trip? 80
What's the maximum temperature you'd like for your trip? 85


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Portland,US,2021-09-25 02:36:17,45.5234,-122.6762,81.18,42,1,1.01
5,5,Ankang,CN,2021-09-25 02:46:32,32.6800,109.0172,81.82,74,96,3.15
6,6,Keti Bandar,PK,2021-09-25 02:46:33,24.1447,67.4497,84.38,76,78,6.02
11,11,Kourou,GF,2021-09-25 02:46:33,5.1552,-52.6478,83.10,89,89,6.51
18,18,Kahului,US,2021-09-25 02:37:12,20.8947,-156.4700,84.22,70,40,20.71
31,31,Sadao,TH,2021-09-25 02:46:36,6.6388,100.4234,83.73,61,95,3.85
40,40,Kavieng,PG,2021-09-25 02:39:41,-2.5744,150.7967,83.66,72,14,10.20
43,43,Kapaa,US,2021-09-25 02:38:14,22.0752,-159.3190,82.38,76,75,3.00
49,49,Maceio,BR,2021-09-25 02:40:30,-9.6658,-35.7353,80.04,83,75,9.22
52,52,Champerico,GT,2021-09-25 02:46:38,14.3000,-91.9167,80.35,88,100,5.82


In [9]:
preferred_cities_df.count()

City_ID       84
City          84
Country       84
Date          84
Lat           84
Lng           84
Max Temp      84
Humidity      84
Cloudiness    84
Wind Speed    84
dtype: int64

In [10]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.isnull().sum()

84


City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df=preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_preferred_cities_df))
clean_preferred_cities_df.head()

84


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Portland,US,2021-09-25 02:36:17,45.5234,-122.6762,81.18,42,1,1.01
5,5,Ankang,CN,2021-09-25 02:46:32,32.6800,109.0172,81.82,74,96,3.15
6,6,Keti Bandar,PK,2021-09-25 02:46:33,24.1447,67.4497,84.38,76,78,6.02
11,11,Kourou,GF,2021-09-25 02:46:33,5.1552,-52.6478,83.10,89,89,6.51
18,18,Kahului,US,2021-09-25 02:37:12,20.8947,-156.4700,84.22,70,40,20.71


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Portland,US,81.18,45.5234,-122.6762,
5,Ankang,CN,81.82,32.6800,109.0172,
6,Keti Bandar,PK,84.38,24.1447,67.4497,
11,Kourou,GF,83.10,5.1552,-52.6478,
18,Kahului,US,84.22,20.8947,-156.4700,
31,Sadao,TH,83.73,6.6388,100.4234,
40,Kavieng,PG,83.66,-2.5744,150.7967,
43,Kapaa,US,82.38,22.0752,-159.3190,
49,Maceio,BR,80.04,-9.6658,-35.7353,
52,Champerico,GT,80.35,14.3000,-91.9167,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Portland,US,81.18,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
5,Ankang,CN,81.82,32.6800,109.0172,安琪酒店
6,Keti Bandar,PK,84.38,24.1447,67.4497,Haji laloo memon house
11,Kourou,GF,83.10,5.1552,-52.6478,Lodge TAKINI4 Guesthouse KOUROU FGU
18,Kahului,US,84.22,20.8947,-156.4700,Maui Seaside Hotel
...,...,...,...,...,...,...
654,Tautira,PF,82.42,-17.7333,-149.1500,Pension Oaoa
656,Meulaboh,ID,83.79,4.1363,96.1285,Kost Pria
657,Manggar,ID,84.99,-2.8833,108.2667,Guest Hotel
672,Dalbandin,PK,80.76,28.8947,64.4101,Gidaan Guest House


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. Display the figure
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
#Add a marker layer for each city to the map.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
start = 'Ballina'
end = 'Ballina'
stop1 = 'Biloela'
stop2 = 'Yeppoon'
stop3 = 'Emerald'

vacation_start = clean_hotel_df.loc[clean_hotel_df['City']=='Ballina']
vacation_end = clean_hotel_df.loc[clean_hotel_df['City']=='Ballina']
vacation_stop1 = clean_hotel_df.loc[clean_hotel_df['City']=='Biloela']
vacation_stop2 = clean_hotel_df.loc[clean_hotel_df['City']=='Yeppoon']
vacation_stop3 = clean_hotel_df.loc[clean_hotel_df['City']=='Emerald']

In [20]:

# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[['Lat','Lng']].to_numpy()[0]
end = vacation_end[['Lat','Lng']].to_numpy()[0]
stop1 = vacation_stop1[['Lat','Lng']].to_numpy()[0]
stop2 = vacation_stop2[['Lat','Lng']].to_numpy()[0]
stop3 = vacation_stop3[['Lat','Lng']].to_numpy()[0]

In [21]:

# Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
travel_route = gmaps.directions_layer(start, end, waypoints=[stop1,stop2,stop3], travel_mode="DRIVING" or "BICYCLING" or "WALKING")
fig.add_layer(travel_route)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:

# To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Ballina,AU,82.78,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
1,Biloela,AU,83.79,-24.4167,150.5000,Biloela Silo Motor Inn
2,Yeppoon,AU,80.38,-23.1333,150.7333,Villa Mar Colina
3,Emerald,AU,83.34,-23.5333,148.1667,The Irish Village


In [23]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [24]:
# Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))